In [ ]:
from local_tree_decomp import *
from help_functions import *

from local_hom_count_best import *

In [ ]:
graph = graphs.CompleteBipartiteGraph(1, 2)
target_graph = graphs.CompleteGraph(3)

graph_clr = [1, 1, 1]
target_clr = [1, 1, 1]

# count_homomorphisms_best(graph, target_graph)

count_homomorphisms_best(graph, target_graph, 2, graph_clr, target_clr, colourful=True)

In [ ]:
target_graph

In [ ]:
graph = graphs.CompleteBipartiteGraph(1, 3)
random_graph = graphs.RandomGNP(100, 0.6, seed=42)

%prun count_homomorphisms_best(graph, random_graph)

In [ ]:
%load_ext line_profiler

from local_hom_count_best import count_homomorphisms_best

%lprun -f count_homomorphisms_best(graph, target_graph)

In [ ]:
g = graph

tree_decomp = g.treewidth(certificate=True)
nice_tree_decomp = make_nice_tree_decomposition(g, tree_decomp)
root = sorted(nice_tree_decomp)[0]
dir_labelled_TD = label_nice_tree_decomposition(nice_tree_decomp, root, directed=True)

In [ ]:
dir_labelled_TD

In [ ]:
node_changes(dir_labelled_TD)

In [ ]:
def best_speed_test(graph, target_graph, graph_clr, target_clr):
    # best_result = count_homomorphisms_best(graph, target_graph)
    # best_colour_result = count_homomorphisms_best_colour(graph, target_graph, graph_clr, target_clr)

    print('Best result:')
    %timeit count_homomorphisms_best(graph, target_graph, density_threshold=2)

    print('\nBest colour result:')
    %timeit count_homomorphisms_best(graph, target_graph, 2, graph_clr, target_clr, True)


In [ ]:
import random
random.seed(int(42))

graph = graphs.CompleteBipartiteGraph(1, 2)
# target_graph = graphs.CompleteGraph(3)

graph_clr = [0, 1, 1]
# target_clr = [0, 1, 2]

for i in range(1, 11):
    N = i * 10
    random_graph = graphs.RandomGNP(N, 0.5, seed=42)
    random_clr = [random.randint(0, 1) for _ in range(N)]
    print('N: ', N)
    best_speed_test(graph, random_graph, graph_clr, random_clr)
    print()

In [ ]:
import random

graph = graphs.CompleteBipartiteGraph(1, 3)
graph_clr = [0, 1, 1, 1]

target_graph = graphs.PetersenGraph()

max_num = 0

for _ in range(500):
    random_clr = [random.randint(0, 1) for _ in range(10)]
    num = count_homomorphisms_best(graph, target_graph, graph_clr, random_clr, colourful=True)
    if num > max_num:
        max_num = num

print(max_num)